加入后验信息的结果比较好，但能不能用是个问题

主要考虑过的特征工程方法有:1、foundation_type可以进行合并，分为两类；

2.数值变量可以考虑做一定的变换，如对数变换、平方根变换、幂变换

(效果都不好）

In [41]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns

In [42]:
raw_structure_data = pd.read_csv('csv_building_structure.csv')

In [43]:
raw_structure_data.dropna(inplace=True)
raw_structure_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 762094 entries, 0 to 762105
Data columns (total 31 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   building_id                             762094 non-null  int64 
 1   district_id                             762094 non-null  int64 
 2   vdcmun_id                               762094 non-null  int64 
 3   ward_id                                 762094 non-null  int64 
 4   count_floors_pre_eq                     762094 non-null  int64 
 5   count_floors_post_eq                    762094 non-null  int64 
 6   age_building                            762094 non-null  int64 
 7   plinth_area_sq_ft                       762094 non-null  int64 
 8   height_ft_pre_eq                        762094 non-null  int64 
 9   height_ft_post_eq                       762094 non-null  int64 
 10  land_surface_condition                  762094 non-null 

In [44]:
distrct_map = pd.read_csv("ward_vdcmun_district_name_mapping.csv")
structure_data = pd.merge(raw_structure_data, distrct_map, on=['ward_id', 'vdcmun_id', 'district_id'], how='left')

In [45]:
structure_data = structure_data.drop(['building_id', 'district_id', 'vdcmun_id', 'ward_id','condition_post_eq', 'technical_solution_proposed', 'vdcmun_name'], axis=1)

In [46]:
structure_data['damage_grade'].value_counts()

Grade 5    275766
Grade 4    183844
Grade 3    136412
Grade 2     87257
Grade 1     78815
Name: damage_grade, dtype: int64

In [47]:
structure_data['net_rooms'] = structure_data.count_floors_post_eq - structure_data.count_floors_pre_eq
structure_data['net_height'] = structure_data.height_ft_post_eq - structure_data.height_ft_pre_eq

In [48]:
structure_data = structure_data.drop(['count_floors_post_eq','count_floors_pre_eq','height_ft_post_eq','height_ft_post_eq'], axis=1)

In [49]:
structure_data['foundation_type'] = structure_data['foundation_type'].replace(['Bamboo/Timber', 'Cement-Stone/Brick', 'RC'], 'Other')

In [50]:
severity_mapping = {'Grade 1': 1, 'Grade 2': 2, 'Grade 3': 3, 'Grade 4': 4, 'Grade 5': 5}
structure_data['damage_grade'] = structure_data['damage_grade'].map(severity_mapping)

In [51]:
columns_to_encode = ['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type',
                      'other_floor_type', 'position', 'plan_configuration']

structure_data = pd.get_dummies(structure_data, columns=columns_to_encode)

In [52]:
structure_data['district_name'].value_counts()

Kavrepalanchok    98019
Makwanpur         90994
Dhading           89122
Sindhupalchok     88741
Gorkha            78074
Nuwakot           77148
Sindhuli          68749
Dolakha           60639
Ramechhap         58612
Okhaldhunga       39352
Rasuwa            12644
Name: district_name, dtype: int64

In [53]:
grouped_dataframes = structure_data.groupby('district_name')
district_dataframes = {name: group for name, group in grouped_dataframes}

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
scaler = StandardScaler()

for name, group in grouped_dataframes:
    X = group.drop(['district_name', 'damage_grade'], axis=1)
    y = group['damage_grade']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    rf_classifier = RandomForestClassifier(random_state=42, n_estimators=1000, max_depth=3, n_jobs=-1)
    rf_classifier.fit(X_train_scaled, y_train)

    y_pred = rf_classifier.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy of {name}: {accuracy:.2f}')


Accuracy of Dhading: 0.68
Accuracy of Dolakha: 0.76
Accuracy of Gorkha: 0.65
Accuracy of Kavrepalanchok: 0.56
Accuracy of Makwanpur: 0.50
Accuracy of Nuwakot: 0.77
Accuracy of Okhaldhunga: 0.44
Accuracy of Ramechhap: 0.54
Accuracy of Rasuwa: 0.80
Accuracy of Sindhuli: 0.45
Accuracy of Sindhupalchok: 0.82
